## Initial EDA

In [25]:
# import libraries needed

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

# Boosting algorithms
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier



In [26]:
# load all 4 datasets to explore individually before combining them

contract_df = pd.read_csv('/Users/oliviarohm/Library/Mobile Documents/com~apple~CloudDocs/Desktop/tripleten/my-portfolio/final_project/final_provider/contract.csv')
personal_df = pd.read_csv('/Users/oliviarohm/Library/Mobile Documents/com~apple~CloudDocs/Desktop/tripleten/my-portfolio/final_project/final_provider/personal.csv')
internet_df = pd.read_csv('/Users/oliviarohm/Library/Mobile Documents/com~apple~CloudDocs/Desktop/tripleten/my-portfolio/final_project/final_provider/internet.csv')
phone_df = pd.read_csv('/Users/oliviarohm/Library/Mobile Documents/com~apple~CloudDocs/Desktop/tripleten/my-portfolio/final_project/final_provider/phone.csv')

In [27]:
# Function for comprehensive dataset overview
def explore_dataset(df, name):
    print(f"{name.upper()}:")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"Data types:\n{df.dtypes}")
    print(f"Missing values:\n{df.isnull().sum()}")
    print(f"Sample data:\n{df.head()}")
    print("-" * 50)

# Apply to each dataset
explore_dataset(contract_df, "contract")
explore_dataset(personal_df, "personal")
explore_dataset(internet_df, "internet")
explore_dataset(phone_df, "phone")

CONTRACT:
Shape: (7043, 8)
Columns: ['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
Data types:
customerID           object
BeginDate            object
EndDate              object
Type                 object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
dtype: object
Missing values:
customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64
Sample data:
   customerID   BeginDate              EndDate            Type  \
0  7590-VHVEG  2020-01-01                   No  Month-to-month   
1  5575-GNVDE  2017-04-01                   No        One year   
2  3668-QPYBK  2019-10-01  2019-12-01 00:00:00  Month-to-month   
3  7795-CFOCW  2016-05-01                   No        One year   
4  9237-HQITU  2019-09-01  2019-11-01 00:00:00  

**Overall Observations:**

- No missing values in any of the datasets
- All datasets contain the same 'customerID' column (dtype: object)
- Column naming inconsistencies identified:
  - `gender`: lowercase
  - `customerID`: camelCase  
  - Others: PascalCase (`SeniorCitizen`, `Partner`, etc.)
  - **Action**: Standardize to snake_case (lowercase + underscores)
 
**Target Variable 'EndDate':**

- Found **only** in **contract_df**
    - 'No' = Still active (not churned)
    - Date = Left on that date (churned)
- **This defines our binary classification target**
    - **Next step:** Convert to binary (1=churned, 0=retained)


**Different Row Counts:**

- Contract: 7,043 customers (100% - baseline)
- Personal: 7,043 customers (100% - complete demographic data)  
- Internet: 5,517 customers (78.3% - subset with internet service)
- Phone: 6,361 customers (90.3% - subset with phone service)
    - **Not all customers have all services**
    - **Merge strategy: LEFT join on contract_df to retain all customers**

**Data Type Issues to Fix:**

- **BeginDate & EndDate**: Currently object, need datetime
    - Required to calculate customer tenure (key feature)
- **TotalCharges**: Currently object, need float64
    - Likely contains empty strings or spaces, "N/A" or similar text values, special characters or formatting issues 
    - Must investigate before conversion
  
**Contract Types:**

- Month-to-month
- One year
- Two year
    - **Hypothesis**: Contract length likely correlates with churn rate
- **Next**: Verify exact values and explore churn patterns by contract type

## Immediate Next Steps:
1. Investigate data type issues:
   - Examine TotalCharges values
   - Check EndDate format variations
2. Fix data types (BeginDate, EndDate, TotalCharges)
3. Create binary churn target from EndDate
4. Standardize column names to snake_case
5. Perform LEFT joins to create master dataset

### 1. Investigate Data Type Issues

In [28]:
# Investigate TotalCharges

print("TotalCharges unique value count:", contract_df['TotalCharges'].nunique())
print("\nSample of TotalCharges values:")
print(contract_df['TotalCharges'].value_counts().head(10))
print("\nChecking for empty/whitespace:")
print(contract_df['TotalCharges'].str.strip().value_counts().head(10))

TotalCharges unique value count: 6531

Sample of TotalCharges values:
TotalCharges
         11
20.2     11
19.75     9
20.05     8
19.9      8
19.65     8
45.3      7
19.55     7
20.15     6
20.25     6
Name: count, dtype: int64

Checking for empty/whitespace:
TotalCharges
         11
20.2     11
19.75     9
20.05     8
19.9      8
19.65     8
45.3      7
19.55     7
20.15     6
20.25     6
Name: count, dtype: int64


**Identified 11 empty string values causing the datatype issue with this column. We need more context as to what those represent in the business context, as it could be new customers, data entry errors, or something else. We will investigate those specific customers before handling them.** 

In [29]:
# Look at customers with empty TotalCharges
empty_charges = contract_df[contract_df['TotalCharges'] == '']
print("Customers with empty TotalCharges:")
print(empty_charges[['customerID', 'BeginDate', 'EndDate', 'TotalCharges']].head())

Customers with empty TotalCharges:
Empty DataFrame
Columns: [customerID, BeginDate, EndDate, TotalCharges]
Index: []


**Strange that it shows no customers with empty string TotalCharges.** Will investigate further.

In [30]:
# Recheck the data type
print(f"\nTotalCharges data type: {contract_df['TotalCharges'].dtype}")

# Look at some unique values
print(f"\nFirst 20 unique values:")
print(contract_df['TotalCharges'].unique()[:20])

# Check for different types of empty/missing values
print("Checking for various empty patterns:")
print(f"Empty strings (''): {(contract_df['TotalCharges'] == '').sum()}")
print(f"Whitespace only: {(contract_df['TotalCharges'].str.strip() == '').sum()}")
print(f"NaN values: {contract_df['TotalCharges'].isna().sum()}")
print(f"None values: {(contract_df['TotalCharges'] == 'None').sum()}")



TotalCharges data type: object

First 20 unique values:
['29.85' '1889.5' '108.15' '1840.75' '151.65' '820.5' '1949.4' '301.9'
 '3046.05' '3487.95' '587.45' '326.8' '5681.1' '5036.3' '2686.05'
 '7895.15' '1022.95' '7382.25' '528.35' '1862.9']
Checking for various empty patterns:
Empty strings (''): 0
Whitespace only: 11
NaN values: 0
None values: 0


In [31]:
# Now that we identified that 11 customers contain whitespace-only characters, we can further investigate

# Find customers with whitespace TotalCharges
whitespace_mask = contract_df['TotalCharges'].str.strip() == ''
empty_charges = contract_df[whitespace_mask]

print(f"Total customers with whitespace TotalCharges: {len(empty_charges)}")
print("\nTheir details:")
print(empty_charges[['customerID', 'BeginDate', 'EndDate', 'MonthlyCharges', 'TotalCharges', 'Type']])

# Check their tenure
print("\nBeginDate for these customers:")
print(empty_charges['BeginDate'].value_counts())

# Churn status
print("\nEndDate (churn status):")
print(empty_charges['EndDate'].value_counts())

Total customers with whitespace TotalCharges: 11

Their details:
      customerID   BeginDate EndDate  MonthlyCharges TotalCharges      Type
488   4472-LVYGI  2020-02-01      No           52.55               Two year
753   3115-CZMZD  2020-02-01      No           20.25               Two year
936   5709-LVOEQ  2020-02-01      No           80.85               Two year
1082  4367-NUYAO  2020-02-01      No           25.75               Two year
1340  1371-DWPAZ  2020-02-01      No           56.05               Two year
3331  7644-OMVMY  2020-02-01      No           19.85               Two year
3826  3213-VVOLG  2020-02-01      No           25.35               Two year
4380  2520-SGTTA  2020-02-01      No           20.00               Two year
5218  2923-ARZLG  2020-02-01      No           19.70               One year
6670  4075-WKNIU  2020-02-01      No           73.35               Two year
6754  2775-SEFEE  2020-02-01      No           61.90               Two year

BeginDate for these cu

**TotalCharges Data Quality Findings:** all 11 customers started on the exact same data (**2020-02-01**) and are all still **active**. All are **long-term contracts**, mostly two years. They all show monthly charges, but whitespace for total charges, which might indicate that their rates have been set, but their first billing cycle hasn't completed yet - which would be explained if the data was collected shortly after (maybe within the same month), of their new contracts. 

**Next step for handling these values:** we will convert these 11 customers to **0.0** to represent **"no charges billed yet"** as it preserves these customers as active, accurately reflects their billing status, and won't skew our analysis with artificial NaN values.

In [32]:
# Replace whitespace with 0 for new customers (not yet billed)
contract_df['TotalCharges'] = contract_df['TotalCharges'].str.strip().replace('', '0')
contract_df['TotalCharges'] = contract_df['TotalCharges'].astype(float)

# Verify
print("TotalCharges fixed:")
print(f"Data type: {contract_df['TotalCharges'].dtype}")
print(f"Zero values (new customers): {(contract_df['TotalCharges'] == 0).sum()}")
print(f"Missing values: {contract_df['TotalCharges'].isna().sum()}")

TotalCharges fixed:
Data type: float64
Zero values (new customers): 11
Missing values: 0


**Investigate BeginDate and EndDate Columns**

In [33]:
# Check BeginDate values
print("BeginDate investigation:")
print(f"Unique values: {contract_df['BeginDate'].nunique()}")
print(f"\nSample values:")
print(contract_df['BeginDate'].value_counts().head(10))
print(f"\nDate range:")
print(f"First: {contract_df['BeginDate'].min()}")
print(f"Last: {contract_df['BeginDate'].max()}")

print("\n" + "="*50)

#Check EndDate values
print("\nEndDate investigation:")
print(f"Unique values: {contract_df['EndDate'].nunique()}")
print(f"\nValue counts:")
print(contract_df['EndDate'].value_counts().head(10))
print(f"\nDate range:")
print(f"First: {contract_df['EndDate'].min()}")
print(f"Last: {contract_df['EndDate'].max()}")
print(f"\nHow many 'No' (active customers):")
print((contract_df['EndDate'] == 'No').sum())
print(f"\nHow many dates (churned customers):")
print((contract_df['EndDate'] != 'No').sum())


BeginDate investigation:
Unique values: 77

Sample values:
BeginDate
2014-02-01    366
2019-10-01    237
2019-11-01    237
2019-09-01    237
2020-01-01    233
2019-12-01    220
2014-03-01    178
2019-07-01    156
2019-08-01    146
2019-06-01    141
Name: count, dtype: int64

Date range:
First: 2013-10-01
Last: 2020-02-01


EndDate investigation:
Unique values: 5

Value counts:
EndDate
No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: count, dtype: int64

Date range:
First: 2019-10-01 00:00:00
Last: No

How many 'No' (active customers):
5174

How many dates (churned customers):
1869



#### **Date Column Investigation:**

All BeginDate values are in consistent date format. EndDate contains actual dates plus 'No' (active customers with no end date yet), which will cleanly convert to NaT ("Not a Time") using `errors='coerce'`.

**Key Observations:**

**BeginDate:**
- Clean date format spanning 2013-10-01 to 2020-02-01 (6+ years)
- 77 unique start dates
- Confirms dataset captured before February 2020 billing cycle completed

**EndDate:**
- Only 5 unique values: 4 churn dates + 'No' (active)
- All churn concentrated in 4-month window: Oct 2019 - Jan 2020
- Much narrower time window than BeginDate

**Class Balance:**
- Active: 5,174 customers (73.5%)
- Churned: 1,869 customers (26.5%)
- Moderate imbalance, appropriate for AUC-ROC metric

**Implications:**
- **DateTime Conversion:** Convert EndDate to datetime format ('No' becomes NaT)
- **Target Variable Creation:** Separate binary target column (NaT → 0 retained, dates → 1 churned)
- **Data Limitation:** Narrow churn window (4 months) vs wide tenure range (6+ years)
  - Question for investigation: What caused concentrated churn Oct 2019-Jan 2020? (Economic factors? Competitor? Policy change?)
  - Model will predict churn patterns specific to this period, may not generalize to other market conditions

**Next Step:** Convert date columns to datetime format


### 2. Date Conversion

In [34]:
# Convert date columns to datetime
contract_df['BeginDate'] = pd.to_datetime(contract_df['BeginDate'])
contract_df['EndDate'] = pd.to_datetime(contract_df['EndDate'], errors='coerce')

# Verify conversion
print("Date conversions complete:")
print(f"BeginDate dtype: {contract_df['BeginDate'].dtype}")
print(f"EndDate dtype: {contract_df['EndDate'].dtype}")

print(f"\nActive customers (EndDate = NaT): {contract_df['EndDate'].isna().sum()}")
print(f"Churned customers (EndDate = date): {contract_df['EndDate'].notna().sum()}")

print("\nSample data:")
print(contract_df[['customerID', 'BeginDate', 'EndDate']].head(10))

Date conversions complete:
BeginDate dtype: datetime64[ns]
EndDate dtype: datetime64[ns]

Active customers (EndDate = NaT): 5174
Churned customers (EndDate = date): 1869

Sample data:
   customerID  BeginDate    EndDate
0  7590-VHVEG 2020-01-01        NaT
1  5575-GNVDE 2017-04-01        NaT
2  3668-QPYBK 2019-10-01 2019-12-01
3  7795-CFOCW 2016-05-01        NaT
4  9237-HQITU 2019-09-01 2019-11-01
5  9305-CDSKC 2019-03-01 2019-11-01
6  1452-KIOVK 2018-04-01        NaT
7  6713-OKOMC 2019-04-01        NaT
8  7892-POOKP 2017-07-01 2019-11-01
9  6388-TABGU 2014-12-01        NaT


**Date Conversion Complete:**
- BeginDate & EndDate successfully converted to datetime64[ns]
- Active customers ('No') → NaT (5,174)
- Churned customers (dates) → datetime (1,869)

### 3. Create Binary Churn Target from EndDate

In [35]:
# Create binary churn target
# 1 = churned (has EndDate), 0 = active (NaT)
contract_df['churn'] = contract_df['EndDate'].notna().astype(int)

# Verify
print("Churn target created:")
print(f"Churned (1): {(contract_df['churn'] == 1).sum()}")
print(f"Retained (0): {(contract_df['churn'] == 0).sum()}")
print(f"\nChurn rate: {contract_df['churn'].mean():.1%}")

# Show sample
print("\nSample data:")
print(contract_df[['customerID', 'EndDate', 'churn']].head(10))

Churn target created:
Churned (1): 1869
Retained (0): 5174

Churn rate: 26.5%

Sample data:
   customerID    EndDate  churn
0  7590-VHVEG        NaT      0
1  5575-GNVDE        NaT      0
2  3668-QPYBK 2019-12-01      1
3  7795-CFOCW        NaT      0
4  9237-HQITU 2019-11-01      1
5  9305-CDSKC 2019-11-01      1
6  1452-KIOVK        NaT      0
7  6713-OKOMC        NaT      0
8  7892-POOKP 2019-11-01      1
9  6388-TABGU        NaT      0


#### Binary Target Variable Created Successfully:

- **Column name:** `churn`
- **Logic Implemented:** EndDate = NaT → 1 (churned), EndDate = valid dates → 0 (active)
- **Distribution:**
  - **Churned (1):** 1,869 customers (26.5%)
  - **Retained (0):** 5,174 customers (73.5%)
- **Modeling note:** Moderate class imbalance is manageable and realistic for churn prediction

### Standardize Columns and Perform Left Join on 'contract_df'
Will visualize data after this step is complete

In [36]:
import re

# Function to convert to snake_case
def to_snake_case(name):
    s1 = re.sub('([a-z])([A-Z])', r'\1_\2', name)
    return s1.lower()

# Function to standardize all columns in a dataframe
def standardize_columns(df):
    df.columns = [to_snake_case(col) for col in df.columns]
    return df

# Test function with actual column names from your datasets
test_columns = ['customerID', 'BeginDate', 'EndDate', 'PaperlessBilling', 
                'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen',
                'InternetService', 'OnlineSecurity', 'MultipleLines']

print("Testing column name conversions:")
for col in test_columns:
    converted = to_snake_case(col)
    print(f"{col:20} -> {converted}")

Testing column name conversions:
customerID           -> customer_id
BeginDate            -> begin_date
EndDate              -> end_date
PaperlessBilling     -> paperless_billing
PaymentMethod        -> payment_method
MonthlyCharges       -> monthly_charges
TotalCharges         -> total_charges
SeniorCitizen        -> senior_citizen
InternetService      -> internet_service
OnlineSecurity       -> online_security
MultipleLines        -> multiple_lines


**Should quickly confirm the customer_id column values are consistent across all four datasets**

In [37]:

# Apply to all dataframes
contract_df = standardize_columns(contract_df)
personal_df = standardize_columns(personal_df)
internet_df = standardize_columns(internet_df)
phone_df = standardize_columns(phone_df)

# Verify
print("CONTRACT_DF columns:", list(contract_df.columns))
print("PERSONAL_DF columns:", list(personal_df.columns))
print("INTERNET_DF columns:", list(internet_df.columns))
print("PHONE_DF columns:", list(phone_df.columns))

# Check if all customer IDs from other datasets exist in contract_df
print("Customer ID overlap check:")
print(f"Contract customers: {len(contract_df['customer_id'].unique())}")
print(f"Personal customers: {len(personal_df['customer_id'].unique())}")
print(f"Internet customers: {len(internet_df['customer_id'].unique())}")
print(f"Phone customers: {len(phone_df['customer_id'].unique())}")


CONTRACT_DF columns: ['customer_id', 'begin_date', 'end_date', 'type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'churn']
PERSONAL_DF columns: ['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents']
INTERNET_DF columns: ['customer_id', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']
PHONE_DF columns: ['customer_id', 'multiple_lines']
Customer ID overlap check:
Contract customers: 7043
Personal customers: 7043
Internet customers: 5517
Phone customers: 6361


#### Successfully standardized column names to snake_case & confirmed customer_id overlap as join key across datasets

**Questions to consider before merging:**

1. How should we handle customers without internet/phone services?
   - Include with "No service" indicators?
   - Or exclude (different churn patterns)?

2. Do churn rates differ by service type?
   - If yes → service availability is an important feature
   - If no → may not matter, but more data is better

**Next:** Investigate churn rates by service type before deciding merge strategy

In [38]:
# Get full picture of how service types relate to churn patterns

# Check churn patterns by service availability
print("Churn analysis by service type:")
print(f"Overall churn rate: {contract_df['churn'].mean():.1%}")

# Customers with internet service
internet_customers = contract_df['customer_id'].isin(internet_df['customer_id'])
print(f"Internet customers churn rate: {contract_df[internet_customers]['churn'].mean():.1%}")

# Customers with phone service  
phone_customers = contract_df['customer_id'].isin(phone_df['customer_id'])
print(f"Phone customers churn rate: {contract_df[phone_customers]['churn'].mean():.1%}")

# Check customers with NO services
no_internet = ~internet_customers
no_phone = ~phone_customers

print(f"No internet service churn rate: {contract_df[no_internet]['churn'].mean():.1%}")
print(f"No phone service churn rate: {contract_df[no_phone]['churn'].mean():.1%}")

# Customers with BOTH services
both_services = internet_customers & phone_customers
print(f"Both services churn rate: {contract_df[both_services]['churn'].mean():.1%}")

Churn analysis by service type:
Overall churn rate: 26.5%
Internet customers churn rate: 31.8%
Phone customers churn rate: 26.7%
No internet service churn rate: 7.4%
No phone service churn rate: 24.9%
Both services churn rate: 32.8%


**Churn Rate by Service Type:**
- No internet: 7.4% (lowest - very loyal)
- No phone: 24.9%
- Internet customers: 31.8%
- Both services: 32.8% (highest)

**Conclusion:** Service availability strongly correlates with churn. **Include all customers** - "No service" is valuable predictive information.

In [39]:
# LEFT join all datasets on customer_id
df = contract_df.merge(personal_df, on='customer_id', how='left')
df = df.merge(internet_df, on='customer_id', how='left')
df = df.merge(phone_df, on='customer_id', how='left')

# Verify
print(f"Final dataset shape: {df.shape}")
print(f"\nMissing values after merge:")
print(df.isnull().sum())

Final dataset shape: (7043, 21)

Missing values after merge:
customer_id             0
begin_date              0
end_date             5174
type                    0
paperless_billing       0
payment_method          0
monthly_charges         0
total_charges           0
churn                   0
gender                  0
senior_citizen          0
partner                 0
dependents              0
internet_service     1526
online_security      1526
online_backup        1526
device_protection    1526
tech_support         1526
streaming_tv         1526
streaming_movies     1526
multiple_lines        682
dtype: int64


**Shape confirms all customers were preserved (7,043 rows), and all features were combined (21 columns)** 

**Missing values as expected:**
- 1,526 customers without internet service (missing internet-related columns)
- 682 customers without phone service (missing multiple_lines column)
- 5,174 customers with missing end_date ("new/NaT customers" - likely meaning they don't have an end date because they're still **active customers**)

**Strategy for missing values:** Because they are **meaningful** to business insights, we will give them meaningful descriptive labels. However, we will **drop end_date column** - as it's already encoded in churn, and keeping it could cause data leakage.

In [40]:
# Fill missing internet-related columns with 'No internet service'
internet_cols = ['internet_service', 'online_security', 'online_backup', 
                 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']
df[internet_cols] = df[internet_cols].fillna('No internet service')

# Fill missing phone column with 'No phone service'
df['multiple_lines'] = df['multiple_lines'].fillna('No phone service')

# Verify
print("Missing values after filling:")
print(df.isnull().sum())
print(f"\nInternet service value counts:")

print(df['internet_service'].value_counts())
print(f"\nMultiple lines value counts:")
print(df['multiple_lines'].value_counts())


Missing values after filling:
customer_id             0
begin_date              0
end_date             5174
type                    0
paperless_billing       0
payment_method          0
monthly_charges         0
total_charges           0
churn                   0
gender                  0
senior_citizen          0
partner                 0
dependents              0
internet_service        0
online_security         0
online_backup           0
device_protection       0
tech_support            0
streaming_tv            0
streaming_movies        0
multiple_lines          0
dtype: int64

Internet service value counts:
internet_service
Fiber optic            3096
DSL                    2421
No internet service    1526
Name: count, dtype: int64

Multiple lines value counts:
multiple_lines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64


In [41]:
# Drop end_date (already encoded as churn target)
df = df.drop(columns=['end_date'])

# Verify
print(f"Shape after dropping end_date: {df.shape}")
print(f"\nRemaining columns: {list(df.columns)}")
print(f"\nMissing values:")
print(df.isnull().sum())

Shape after dropping end_date: (7043, 20)

Remaining columns: ['customer_id', 'begin_date', 'type', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'churn', 'gender', 'senior_citizen', 'partner', 'dependents', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'multiple_lines']

Missing values:
customer_id          0
begin_date           0
type                 0
paperless_billing    0
payment_method       0
monthly_charges      0
total_charges        0
churn                0
gender               0
senior_citizen       0
partner              0
dependents           0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_tv         0
streaming_movies     0
multiple_lines       0
dtype: int64


**Data Cleaning Complete:**
- Dropped end_date (already encoded as churn target - avoids data leakage)
- Final shape: 7,043 × 20
- Zero missing values
- Customer information (customer_id, begin_date, gender, senior_citizen, etc.)
- Service details (internet_service, multiple_lines, streaming services, etc.)
- Contract information (type, payment_method, paperless_billing)
- Financial data (monthly_charges, total_charges)

## Clarifying Questions

**Data & Context:**
1. Why is churn data concentrated in a 4-month window (Oct 2019 - Jan 2020)?
2. Are there seasonal trends in customer behavior that might explain this timing?
3. Are there any known business changes during this period (pricing, competitors, policy)?

**Business Goals:**

4. What customer characteristics are most associated with churn?
5. Which contract types and payment methods lead to higher churn?
7. How does service bundling affect churn likelihood?
9. Is there a customer tenure "danger zone" where churn risk is highest?

**Business Impact:**

8. What's the cost of acquiring a new customer vs. retaining an existing one?
9. What retention strategies are currently in place, and how effective are they?

**Operational:**

10. What actions can the business take based on model predictions?
11. What's the acceptable false positive/negative rate for churn predictions?

**Model Deployment:**

12. How frequently should the model be retrained?
13. What's the timeline for implementing churn prevention actions?

**Technical:**

14. Given the narrow churn window, will this model generalize to other time periods?
15. Are there data privacy constraints on using certain customer features?

---

## Work Plan

**Step 1: Data Preprocessing**
Prepare the merged dataset by handling data types, creating the binary churn target, standardizing column names, and ensuring data quality.

**Step 2: Feature Engineering**
Create new features including:
- Customer tenure (months from begin_date)
- Service bundling indicators
- Charge ratios (monthly vs. total)
- Encode categorical variables for modeling

**Step 3: Exploratory Data Analysis (EDA)**
- Univariate analysis: distributions, outliers
- Bivariate analysis: feature vs. churn relationships
- Correlation analysis among numeric features
- Validate hypotheses (contract type, tenure, charges vs. churn)

**Step 4: Model Building**
- Split data into train/validation/test sets (60/20/20) to prevent overfitting
- Establish baseline model (Logistic Regression)
- Train boosting algorithms (handle mixed data types well, provide feature importance, robust to outliers)
- Use validation set for hyperparameter tuning and model selection
- Evaluate final model on held-out test set

**Step 5: Evaluation & Conclusions**
- Primary metric: AUC-ROC
- Secondary metrics: Accuracy, Precision/Recall
- Analyze feature importance for business insights
- Document model limitations and recommendations